<a href="https://colab.research.google.com/github/KW-plato/DriftDiffusionModel/blob/main/fit_HDDM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Estimate DDM parameters using HDDM 0.6**

code adapted from Anne Urai's fitHDDM.py script https://github.com/anne-urai/2019_Urai_choice-history-ddm/blob/master/fitHDDM.py

In [ ]:
#!apt-get install python3.5

In [ ]:
#!pip install pymc

In [ ]:
#!pip install pandas patsy

In [ ]:
#!pip install kabuki

In [ ]:
#!pip install hddm

In [ ]:
import glob
import hddm
import kabuki
import numpy as np
import os
import pandas as pd
import time

In [ ]:
print(hddm.__version__)

0.8.0


In [ ]:
def recode_4stimcoding(mydata):
    #code stimulus and response direction left as 0, leave direction right as 1.
    
    mydata.loc[mydata['direction']==-1,'direction'] = 0
    mydata.loc[mydata['response']==-1,'response'] = 0
    for col in mydata.columns.tolist():
        if ('stim' in col) or ('resp' in col):
            mydata.loc[mydata[col]==-1,col] = 0
    
    return mydata

def z_link_func(x):
    return 1 / (1 + np.exp(-(x.values.ravel())))

def aic(self):
    k = len(self.get_stochastics())
    logp = sum([x.logp for x in self.get_observeds()['node']])
    return 2 * k - 2 * logp


def bic(self):
    k = len(self.get_stochastics())
    n = len(self.data)
    logp = sum([x.logp for x in self.get_observeds()['node']])
    return -2 * logp + k * np.log(n)

def concat_models(mypath, model_name, nchains=30):
    traces = range(1,nchains+1)

    # CHECK IF COMBINED MODEL EXISTS
    if os.path.isfile(os.path.join(mypath, model_name, 'modelfit-combined.model')):
        print("Combined Model exists: {}".format(os.path.join(mypath, model_name, 'modelfit-combined.model')))
    else:
        # ============================================ #
        # APPEND MODELS
        # ============================================ #
        allmodels = []
        print("Combining all traces for %s" % model_name)
        for trace_id in traces:  # how many chains were run?
            model_filename = os.path.join(mypath, model_name, 'modelfit-md%d.model' % trace_id)
            if os.path.isfile(model_filename) == True:  # if not, this model has to be rerun
                print(model_filename)
                thism = hddm.load(model_filename)
                allmodels.append(thism)  # now append into a list
            else:
                print("Not found: trace_id {:2d}".format(trace_id))
                
        if len(allmodels) != nchains:
            return None
        # ============================================ #
        # CHECK CONVERGENCE if all traces were found
        # ============================================ #    
        print("Performing gelman rubin convergence test\n")
        try:
            gr = hddm.analyze.gelman_rubin(allmodels)
            # save
            text_file = open(os.path.join(mypath, model_name, 'gelman_rubin.txt'), 'w')
            for p in gr.items():
                text_file.write("%s,%s\n" % p)
                # print a warning when non-convergence is detected
                # Values should be close to 1 and not larger than 1.02 which would indicate convergence problems.
                # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3731670/
                if abs(p[1] - 1) > 0.02:
                    print("non-convergence found, %s:%s" % p)
            text_file.close()
            print("written gelman rubin stats to file")
        except Exception as e:
            print("Error: {}".format(e))
        m = kabuki.utils.concat_models(allmodels) #creates one model from all chains

        # ============================================ #
        # SAVE THE FULL MODEL
        # ============================================ #

        m.save(os.path.join(mypath, model_name, 'modelfit-combined.model'))  # save combined modelto disk
        print("Concatenated model saved!")
        
        # ============================================ #
        # SAVE POINT ESTIMATES
        # ============================================ #

        print("saving stats...")
        results = m.gen_stats()  # point estimate for each parameter and subject
        results.to_csv(os.path.join(mypath, model_name, 'results-combined.csv'))

        # save the DIC for this model
        text_file = open(os.path.join(mypath, model_name, 'DIC-combined.txt'), 'w')
        text_file.write("Combined model: {}\n".format(m.dic))
        text_file.close()
        print('done')
        
        # ============================================ #
        # SAVE TRACES
        # ============================================ #

        print("saving traces...")
        # get the names for all nodes that are available here
        group_traces = m.get_group_traces()
        group_traces.to_csv(os.path.join(mypath, model_name, 'group_traces.csv'))

        all_traces = m.get_traces()
        all_traces.to_csv(os.path.join(mypath, model_name, 'all_traces.csv'))
        print('done')
        
        # ============================================ #
        # CONCATENATE MODEL COMPARISON
        # ============================================ #
        # average model comparison values across chains
        print('concatenating model comparison...')
        fls = glob.glob(os.path.join(mypath, model_name, 'model_comparison_md*.csv'))
        tmpdf = pd.concat([pd.read_csv(f) for f in fls ])
        # average over chains
        df2 = tmpdf.mean()
        df2 = tmpdf.describe().loc[['mean']]
        df2.to_csv(os.path.join(mypath, model_name, 'model_comparison.csv')) # save comparison to disk
        print('done')


        # DELETE FILES to save space
        print("Now deleting files for seperate chains...")
        for fl in glob.glob(os.path.join(mypath, model_name, 'modelfit-md*.model')):
            print(fl)
            os.remove(fl)
        for fl in glob.glob(os.path.join(mypath, model_name, 'modelfit-md*.db')):
            if not '-md1.db' in fl: #needed here to load the pickled db in PPC
                print(fl)
                os.remove(fl)
        for fl in glob.glob(os.path.join(mypath, model_name, 'model_comparison_md*.csv')):
            print(fl)
            os.remove(fl)
        for fl in glob.glob(os.path.join(mypath, model_name, 'DIC-md*.txt')):
            print(fl)
            os.remove(fl)
        print('DONE!!!')


In [ ]:
def make_model(mydata, model_name, trace_id, nlag=0):
    
    #checks before model is created
    if "nohist" not in model_name and nlag == 0:
        print("For all models with history effects, 'nlag' must be non-zero")
        exit(0)
    elif "nohist" in model_name and nlag != 0:
        print("'nlag' specified but model is without history effect. 'nlag' value is ignored\n")
        nlag = 0
         
    if 'regress' in model_name:
        if nlag != 0:
            lags = range(1,nlag+1)
            resp_cols = ['l' + str(i) + '_resp' for i in lags]
            stim_cols = ['l' + str(i) + '_stim' for i in lags]
            resps = " + ".join(resp_cols)
            respstim = " + ".join(resp_cols + stim_cols)
            for col in stim_cols:
                mydata = mydata[mydata[col].notna()]
    else:
        mydata = recode_4stimcoding(mydata)
        if nlag != 0:
            col_resp = 'l' + str(nlag) + '_resp'
            col_stim = 'l' + str(nlag) + '_stim'
            col_stim = 'l' + str(nlag) + '_subject'
            col_repeat = 'l' + str(nlag) + '_repeat'
             
    if model_name == 'stimcoding_nohist': # NO HISTORY FOR MODEL COMPARISON
        m = hddm.HDDMStimCoding(mydata, stim_col='direction', split_param='v',
                drift_criterion=True, bias=True, p_outlier=0.05,
                include=('sv', 'sz'), group_only_nodes=['sv', 'sz'])
    elif model_name == 'stimcoding_dc_resp':
        m = hddm.HDDMStimCoding(mydata, stim_col='direction', split_param='v',
                drift_criterion=True, bias=True, p_outlier=0.05,
                include=('sv', 'sz'), group_only_nodes=['sv', 'sz'],
                depends_on={'dc':[col_resp]})
    elif model_name == 'stimcoding_z_resp':
        m = hddm.HDDMStimCoding(mydata, stim_col='direction', split_param='v',
                drift_criterion=True, bias=True, p_outlier=0.05,
                include=('sv', 'sz'), group_only_nodes=['sv', 'sz'],
                depends_on={'z':[col_resp]})
    elif model_name == 'stimcoding_dc_z_resp':
        m = hddm.HDDMStimCoding(mydata, stim_col='direction', split_param='v',
                drift_criterion=True, bias=True, p_outlier=0.05,
                include=('sv', 'sz'), group_only_nodes=['sv', 'sz'],
                depends_on={'dc':[col_resp], 'z':[col_resp]})
    elif model_name == 'stimcoding_dc_z_st_resp': # also estimate across-trial variability in nondecision time
        m = hddm.HDDMStimCoding(mydata, stim_col='direction', split_param='v',
                drift_criterion=True, bias=True, p_outlier=0.05,
                include=('sv', 'sz', 'st'), group_only_nodes=['sv', 'sz', 'st'],
                depends_on={'dc':[col_resp], 'z':[col_resp]})
    # ============================================ #
    # Dyadic models..depends on whether the lagged trial was own for partner's
    # ============================================ #
    elif model_name == 'stimcoding_dc_z_resp_dyadic':
        m = hddm.HDDMStimCoding(mydata, stim_col='direction', split_param='v',
                drift_criterion=True, bias=True, p_outlier=0.05,
                include=('sv', 'sz'), group_only_nodes=['sv', 'sz'],
                depends_on={'dc':[col_resp, col_subject], 'z':[col_resp, col_subject]})
    
    return m

In [ ]:
def get_full_model_name(m,lag):
    return m if 'nohist' in m else m + '_l' + str(lag)

In [ ]:
def run_model(m, mypath, model_name, trace_id, n_samples):
    print("Running {:<s}, trace_id {:2d}".format(model_name,trace_id))
    print("finding starting values")
    try:
        m.find_starting_values() # this should help the sampling
    except Exception as e:
        print(e) #even if starting values couldnt be found, sampling can continue

    print("begin sampling")
    #m.sample(n_samples, burn=n_samples/2, thin=3)
    
    m.sample(n_samples, burn=n_samples/2, thin=3, db='pickle',
        dbname=os.path.join(mypath, model_name, 'modelfit-md%d.db'%trace_id))
    m.save(os.path.join(mypath, model_name, 'modelfit-md%d.model'%trace_id)) # save the model to disk
    
    # ============================================ #
    # save the output values
    # ============================================ #

    # save the DIC for this model
    text_file = open(os.path.join(mypath, model_name, 'DIC-md%d.txt'%trace_id), 'w')
    text_file.write("Model {}: {}\n".format(trace_id, m.dic))
    text_file.close()

    # save the other model comparison indices
    df = dict()
    df['trace_id'] = trace_id
    df['dic_original'] = [m.dic]
    df['aic'] = [aic(m)]
    df['bic'] = [bic(m)]
    df2 = pd.DataFrame(df)
    df2.to_csv(os.path.join(mypath, model_name, 'model_comparison_md%d.csv'%trace_id),index=False)


In [ ]:
def recode_subjidx_lag_trials(df,nprev):
    
    colname = 'l' + str(nprev) + '_subject'
    df[colname] = (df[colname] == df['subj_idx']) * 1
    return df

In [ ]:
models_collection = {
    1: 'stimcoding_nohist',  # no history baseline model
    2: 'stimcoding_dc_resp',  #previous response dependent, nlags needed
    3: 'stimcoding_z_resp',  #previous response dependent, nlags needed
    4: 'stimcoding_dc_z_resp', #previous response dependent, nlags needed
    5: 'stimcoding_dc_z_st_resp',  #previous response dependent, nlags needed
    6: 'stimcoding_dc_z_resp_dyadic' #previous response dependent, nlags needed, dyadic assessment
}


In [ ]:
"""
Prepare for running: Part I
Select model and the nth prev trial intended to be analyzed
"""
model = models_collection[19]
lag = 1
chains = 2
trace_ids = range(1,chains+1)
n_samples = 50

mypath = "output"
full_model_name = get_full_model_name(model,lag)
thispath = os.path.join(mypath,full_model_name)
if not os.path.exists(thispath):
    os.makedirs(thispath)


In [ ]:
"""
Prepare for running:Part II
Setup the folders and fetch data
"""
mypath = "output"
datasrc = "data/coded"
datafile = "all_trials_data.csv"

#if the concatenated data from each experimnetal run exists, use that, else create the concatenated data first
data_loc = os.path.join(datasrc,datafile)
if os.path.exists(data_loc):
    mydata = hddm.load_csv(data_loc)
    print("Concatenated datafile with full experiment data found and loaded")
else:
    files = glob.glob(os.path.join(datasrc,'pair*.csv'))
    mydata = pd.concat([pd.read_csv(f) for f in files ])
    mydata.to_csv(data_loc,index=False,header=True)
    print("Concatenated datafile with full experiment data created")
    

#DDM param estimations are done for 3 conditions: 
# 1. No history condition
# 2. depends on the lagged trial response 
# 3. depends on the lagged trial response and own (code 1) or partner (code 0)trial

mydata = recode_subjidx_lag_trials(mydata,lag)


In [ ]:
# ============================================ #
# Main HDDM parameter estimation
# ============================================ #
starttime = time.time()
for trace_id in trace_ids:
    model_filename = os.path.join(mypath, full_model_name, 'modelfit-md%d.model' % trace_id)
    m = make_model(mydata, model, trace_id, lag)
    run_model(m, mypath, full_model_name, trace_id, n_samples)
    elapsed = time.time() - starttime
    print("\nElapsed time for %s, trace_id %d, %d samples: %f seconds\n" % (model, trace_id, n_samples, elapsed))

concat_models(mypath, full_model_name, chains)

In [ ]:
# ============================================ #
# POSTERIOR PREDICTIVES TO ASSESS MODEL FIT
# ============================================ #

starttime = time.time()
print("computing ppc")
# specify how many samples are needed
m = hddm.load(os.path.join(mypath,full_model_name, 'modelfit-combined.model'))
nsmp = 100
ppc = hddm.utils.post_pred_gen(m, append_data=True, samples=nsmp)

# make the csv smaller, save disk space
savecols = list(set(ppc.columns) & set(['rt','rt_sampled', 'response_sampled',
                        'index', 'direction', 'response', 'l1_resp', 'subj_idx']))
ppc = ppc[savecols]
# save as pandas dataframe
ppc.to_csv(os.path.join(mypath, full_model_name, 'ppc_data.csv'), index=True)
elapsed = time.time() - starttime
print( "\nElapsed time for %s, PPC: %f seconds\n" %(full_model_name,elapsed))

In [ ]:
# ============================================ #
# QUANTILE OPTIMISATION
# http://ski.clps.brown.edu/hddm_docs/howto.html#run-quantile-opimization
# ============================================ #

subj_params = []
bic = []

for subj_idx, subj_data in mydata.groupby('subj_idx'):
    m_subj = make_model(subj_data, model, 1,lag)
    thismodel = m_subj.optimize('gsquare', quantiles=[0.1, 0.3, 0.5, 0.7, 0.9], n_runs=5)
    thismodel.update({'subj_idx':subj_idx}) # keep original subject number
    subj_params.append(thismodel)
    bic.append(m_subj.bic_info)

params = pd.DataFrame(subj_params)
params.to_csv(os.path.join(mypath, full_model_name, 'Gsquare.csv'))
bic = pd.DataFrame(bic)
bic.to_csv(os.path.join(mypath, full_model_name, 'BIC.csv'))
print("QUANTILE OPTIMISATION. DONE!!!")